In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from utils import *
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Helvetica'

In [ ]:
# del hc  # uncomment to reload 
try: print('Histories already loaded.') if hc else None
except: hc = load_histories_and_configs(['ix4onq8c', '9cclxb43'])

_split = make_split_df(hc)
_levels = make_df(hc, 'pruning_levels')
_pr = make_df(hc, 'pruning_rate')
_splitrange = make_splitrange_df(hc)
_first, _last = make_dfs_at_splitrange(hc, _splitrange)

# average over the seeds via pruning_levels (sweep parameters)
ydf = pd.concat([_split, _levels], axis=1).groupby('pruning_levels').sum()
xdf = pd.concat([_pr, _levels], axis=1).groupby('pruning_levels').mean()

In [ ]:
title='Zero to 20 Pruning levels with same target.'
description=(
'''#
Fully connected network in the circles-moons dataset, with shape [4,410,410,2].
Each run, the networks are pruned to the same pruning target of 112 weights.
Increasing the pruning levels, decreases the pruning rate per level.
'''
)

# Create figure and do general styling
fig, axes = plt.subplots(nrows=2, figsize=(16,9), sharey=True, sharex=False)
fig.suptitle(title, fontweight='regular', fontsize=15, fontname='Helvetica')

ydf[splitters.keys()].plot.bar(
    ax=axes[0],
    width=0.95,  
    color=splitters.values(),
    rot=0,
    stacked=True
)
ydf[nonsplitters.keys()].plot.bar(
    ax=axes[1],
    width=0.95,  
    color=nonsplitters.values(),
    rot=0,
    stacked=True
)
# x axis data
xticks = np.arange(len(xdf.index))
xticklabels = [f'{l}\n{pr*100:.1f}%' for l, pr in xdf['pruning_rate'].items()]

for ax in axes:
    ax.set_yticks(np.arange(np.max(ydf)+1))
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticklabels, fontsize=10, color='gray')
    ax.set_ylabel('num models', fontsize=14)
    ax.set_xlabel('hidden-dim\np-rate', fontsize=10, color='gray')
    ax.xaxis.set_label_coords(0.0, -0.024)

plt.tight_layout(
    #rect=[0, 0.03, 1, 0.95]
)
set_style(axes)
plt.savefig("increase-pruning-levels.svg")

In [ ]:
group_df = _levels
source_df = _first
source_keys = [
    'active-weights-abs', 
    'zombious-abs',
    ]

group = make_group(group_df, source_df, source_keys)

for key in source_keys:
    df = mean_std_from_group(group, key)
    errorbar_from_df(df, label=key, marker='o', alpha=0.6)
plt.title('Metrics at split iteration - 20 pruning levels ')
plt.xlabel('number of hidden neurons')

In [ ]:

plt.figure(figsize=(12, 4)) 

group = make_group(_levels, _first, ['active-weights-abs'])
df = mean_std_from_group(group, 'active-weights-abs').reset_index(drop=True)
plt.errorbar(
    x=df.index, 
    y=df['y'], 
    yerr=df['yerr'],
    marker='o',
    alpha=0.7,
    label='active weights (first iter)',
)

group = make_group(_levels, _last, ['active-weights-abs'])
df = mean_std_from_group(group, 'active-weights-abs').reset_index(drop=True)
plt.errorbar(
    x=df.index, 
    y=df['y'], 
    yerr=df['yerr'],
    marker='o',
    alpha=0.7,
    label='active weights (last iter)',
)

plt.title('Networks at first and last split iteration\n(mean, std of 4 runs)')
plt.xticks(ticks=np.arange(len(df['x'])), labels=df['x'])
plt.xlabel('number of hidden neurons')
plt.legend()
plt.savefig("active-first-last-410.svg")

In [ ]:

# Fit with polyfit
x = df.dropna()['x'].values
y = df.dropna()['y'].values
b, m = np.polynomial.polynomial.polyfit(x,y, 1)

plt.plot(x,y, '.')
plt.plot(x, b + m * x, '-')
plt.show()
b, m

In [ ]:
group_df = _levels
source_df = _last
source_df = _first
source_keys = [
    #'pparams',    
    'active-weights-abs'
]
source_labels = [
    #'available parameters', 
    'active parameters']

group = make_group(group_df, source_df, source_keys)

plt.figure(figsize=(12, 4)) 
for key, label in zip(source_keys, source_labels):
    df = mean_std_from_group(group, key).reset_index(drop=True)
    plt.errorbar(
        x=df.index, 
        y=df['y'], 
        yerr=df['yerr'],
        marker='o',
        alpha=0.7,
        label=label,
    )

plt.title('Active weights, first split\n(mean, std of 4 runs)')
plt.xticks(ticks=np.arange(len(df['x'])), labels=df['x'])
plt.xlabel('number of pruning levels')
plt.legend()
plt.savefig("active.svg")

In [ ]:
group_df = _levels
source_df = _first
source_keys = [
    'active-weights-rel', 
    'zombious-rel',
    ]

group = make_group(group_df, source_df, source_keys)

for key in source_keys:
    df = mean_std_from_group(group, key)
    errorbar_from_df(df, label=key, marker='o', alpha=0.6)
plt.title('Metrics at split iteration')
plt.xlabel('number of pruning levels')
plt.xticks(ticks=np.arange(len(df['x'])), labels=df['x'])
plt.show()